In [1]:
from src.datasets.Dataset_PESO import Dataset_PESO
from src.utils.Study import Study
from src.utils.ProjectConfiguration import ProjectConfiguration
from src.utils.BaselineConfigs import EXP_OctreeNCA
from src.datasets.Dataset_BCSS_Seg import Dataset_BCSS_Seg
from src.datasets.Dataset_AGGC import Dataset_AGGC
import octree_vis, torch, os, json, openslide, math
import einops
from src.models.Model_OctreeNCAV2 import OctreeNCAV2
import numpy as np
import torch.nn.functional as F
from src.utils.ProjectConfiguration import ProjectConfiguration as pc
from src.models.Model_OctreeNCA_2d_patching2 import OctreeNCA2DPatch2
import matplotlib.pyplot as plt
import time
import nca_cuda
torch.set_grad_enabled(False)

/home/nlemke/remote/miniconda3/envs/nca3/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/nlemke/remote/miniconda3/envs/nca3/lib/python3.10/site-packages/transformers/utils/generic.py:441: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(


In [2]:
model_path = "/local/scratch/clmn1/octree_study_new/Experiments/pesofFixAbl_none_10_1.0_16_3_1.0_0.99_OctreeNCA2DSegmentation"
with open(os.path.join(model_path, "config.json")) as f:
    config = json.load(f) 

exp = EXP_OctreeNCA().createExperiment(config, detail_config={}, 
                                                      dataset_class=Dataset_PESO, dataset_args={
                                                            'patches_path': os.path.join(pc.FILER_BASE_PATH, config['experiment.dataset.patches_path']),
                                                            'patch_size': config['experiment.dataset.input_size'],
                                                            'path': os.path.join(pc.FILER_BASE_PATH, config['experiment.dataset.img_path']),
                                                            'img_level': config['experiment.dataset.img_level']
                                                      })

model: OctreeNCA2DPatch2 = exp.model
assert isinstance(model, OctreeNCA2DPatch2)
model.eval()

loading patches
Reload State 2000
loading patches
loading patches

-------- Experiment Setup --------
{
    "experiment.name": "pesofFixAbl_none_10_1.0_16_3_1.0_0.99",
    "experiment.description": "OctreeNCA2DSegmentation",
    "model.output_channels": 1,
    "model.channel_n": 16,
    "model.fire_rate": 0.5,
    "model.kernel_size": [
        3,
        3,
        3,
        3,
        3
    ],
    "model.hidden_size": 64,
    "model.batchnorm_track_running_stats": false,
    "model.train.patch_sizes": [
        null,
        null,
        null,
        null,
        null
    ],
    "model.train.loss_weighted_patching": false,
    "model.eval.patch_wise": false,
    "model.octree.res_and_steps": [
        [
            [
                320,
                320
            ],
            10
        ],
        [
            [
                160,
                160
            ],
            10
        ],
        [
            [
                80,
                80
            ],
 

OctreeNCA2DPatch2(
  (backbone_ncas): ModuleList(
    (0-4): 5 x BasicNCA2DFast(
      (fc0): Conv2d(32, 64, kernel_size=(1, 1), stride=(1, 1))
      (fc1): Conv2d(64, 13, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (conv): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=16, padding_mode=reflect)
      (bn): Identity()
    )
  )
)

In [ ]:
torch.cuda.memory._record_memory_history()
state = torch.zeros(1, 16, 320,320, device='cuda')

bb = model.backbone_ncas[0]
const_inputs = state[:,0:bb.input_channels].clone()
# state.shape: BCHW
delta_state = bb.conv(state)
delta_state = torch.cat([state, delta_state], dim=1)
delta_state = bb.fc0(delta_state)
delta_state = bb.bn(delta_state)
delta_state = F.relu(delta_state)
delta_state = bb.fc1(delta_state)

with torch.no_grad():
    stochastic = torch.zeros([delta_state.size(0),1,delta_state.size(2),
                            delta_state.size(3)], device=delta_state.device)
    stochastic.bernoulli_(p=0.5).float()
delta_state = delta_state * stochastic

new_state = state[:, bb.input_channels:] + delta_state
state = torch.cat([const_inputs, new_state], dim=1)
torch.cuda.memory._dump_snapshot("mem_snapshots/traditional_single_step.pickle")

In [3]:
torch.cuda.memory._record_memory_history()
state = torch.zeros(1, 16, 320,320, device='cuda')

state = state.contiguous()

new_state = torch.zeros(state.size(0), state.size(1), state.size(2), state.size(3), device=state.device)
new_state[:, 0].bernoulli_(0.5)
nca_cuda.nca2d_cuda(new_state, state, model.backbone_ncas[0].conv.weight, model.backbone_ncas[0].conv.bias, model.backbone_ncas[0].fc0.weight, model.backbone_ncas[0].fc0.bias, model.backbone_ncas[0].fc1.weight)
#del state
state = new_state
torch.cuda.memory._dump_snapshot("mem_snapshots/cuda_single_step.pickle")